In [11]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

# Change to MPS logic if on mac
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

For this notebook, use the hints to fill in the missing code.

In [28]:
FILL_IN = "FILL IN"

In [12]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
    
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [13]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [14]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [15]:
MAX_LENGTH = 10

# We'll just read in data with these prefixes so we have easier data to deal with
# These are "target" prefixes
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

# Only use pairs where the english data (pair[1]) has the prefix above
# Also, only consider data where pair[0] and pair[1] have length less than MAX_LENGTH
def filterPair(p):
    return FILL_IN

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [16]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['je suis heureuse que vous soyez la .', 'i m happy you re here .']


In [17]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # Initialize the embedding and lstm; use batch_first=True
        self.embedding = FILL_IN
        self.lstm = FILL_IN

    def forward(self, input, hidden_cell):
        # Get the embeddings and reshape to be (1, 1, -1)
        # Why? remember we use batch size = 1 in this HW for simplicity
        embedded = FILL_IN
        output = embedded
        # Push through the lstm
        output, hidden_cell = FILL_IN
        return output, hidden_cell

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [18]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # Initialize the embedding
        self.embedding = FILL_IN
        # Initialize an LSTM with yt and kt dimensions hidden_size
        # Use batch_first=True
        self.lstm = FILL_IN
        # Initialize a Linear layer going to the appropriate vocabulary size
        self.out = FILL_IN
        # Optional: If you use NLLLoss, iniialize LogSoftmax here
        # What dimension?
        self.softmax = FILL_IN

    def forward(self, input, hidden_cell):
        output = self.embedding(input).view(1, 1, -1)
        # Push through relu
        output = FILL_IN
        # Push output and hidden_cell tuple through the lstm
        output, hidden_cell = FILL_IN
        # Apply LogSoftmax to output (?)
        # Note you can leave this out if you wor with logits and CrossEntropyLoss
        output = FILL_IN
        return output, hidden_cell

    def initHidden(self):
        # Used to initialize the hidden state (or cell state) to a tensor of dimension (1, 1, hidden_size)
        # Just return a tensor here
        return FILL_IN

In [19]:
# Split a sentence by ' ' and return a list of the tokens (int ids) for each word
# Use word2index
def indexesFromSentence(lang, sentence):
    return FILL_IN

# Call the above on a sentence
# After calling, add the EOS_token (int id) to the gotten list
# Return a tensor, but reshape it so it's dimensions (-1, 1)
def tensorFromSentence(lang, sentence):
    indexes = FILL_IN
    FILL_IN
    return FILL_IN

# For a source, target pair, call the above. Return a tuple of 2 tensors, one input_tensor and another an output_tensor
def tensorsFromPair(pair):
    input_tensor = FILL_IN
    target_tensor = FILL_IN
    return (input_tensor, target_tensor)

In [20]:
# For 50% of pairs, use teacher forcing so that we predict (y_1, y_2, ..., y_{T}) from (y_0, y_1, ..., y_{T-1}) on the decoder side
# Without techer forcing, we start with y_0 = SOS_token and then use \hat{y}_1, the prediction at time step 0 as the input to time step 1 on the decoder side
# For this case, we'll predict (\hat{y}_1, \hat{y}_2, ..., \hat{y}_{T}) from (y_0, \hat{y}_1, ..., \hat{y}_{T-1}) on the decoder side
# The crucial thing to realize here is that \hat{y}_t is stochastic, and dependent on what the model predicts - mistakes propegate!
teacher_forcing_ratio = 0.5

def train(
    input_tensor,
    target_tensor,
    encoder,
    decoder,
    encoder_optimizer,
    decoder_optimizer,
    criterion,
    max_length=MAX_LENGTH
):
    # Initialize the hidden and cell states
    encoder_hidden = FILL_IN
    encoder_cell = FILL_IN

    # Reset the optimizer gradients to 0
    FILL_IN
    FILL_IN

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0
    
    # Reverse the input tensor, see below
    input_tensor = FILL_IN
        
    # Reverse the input here
    # If we want to predict [x, y, z] from [a, b, c], we should feed in [c, b, a] on the decoder side so that a is as close to x as possible
    # The above trick was noted in making a big difference
    for it in range(input_length):
        encoder_output, (encoder_hidden, encoder_cell) = FILL_IN

    # Initialize the decoder input to the SOS_token
    decoder_input = FILL_IN

    # Initialize the hidden states of the decoder with the hidden states of the encoder
    decoder_hidden = FILL_IN
    decoder_cell = FILL_IN

    # For this pair, use teacher forcing with 50% probability, else don't
    use_teacher_forcing = FILL_IN
    
    target_length_used = 0

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        target_length_used = target_length
        
        for jt in range(target_length):
            # Push decoder_input, decoder_hidden, and decoder_cell through the decoder
            decoder_output, (decoder_hidden, decoder_cell) = FILL_IN
            # Update the loss
            loss += FILL_IN
            # Set the next decoder_input to the current y_t
            decoder_input = FILL_IN  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        # Note that in this case we feed in at most target_length words
        # If, however, we predict EOS_token, we break out
        # You need to also carefully get the target lenght used since it might not be target_length
        for jt in range(target_length):
            # As before
            decoder_output, (decoder_hidden, decoder_cell) = FILL_IN
            
            # Get the top index, \hat{y}_t; this will be the next decoder_input
            topv, topi = FILL_IN
            decoder_input = FILL_IN  # Detach from history as input

            loss += FILL_IN
            
            # Update the target_length_used
            target_length_used += FILL_IN
            
            # If the EOS_token was generated, exit
            FILL_IN

    # Collect gradients
    FILL_IN

    # Do a step; do this both for the encoder and the decoder
    FILL_IN
    FILL_IN

    # Return the loss for this pair. Note that you'll need to use target_length_used not target_length; why?
    return FILL_IN

In [21]:
# Helper functions used to time the optimizations
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

trainIters below does SGD (batch 1) on the data.

In [22]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

def trainIters(
    encoder,
    decoder,
    n_iters,
    print_every=1000,
    plot_every=100,
    learning_rate=0.01
):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    # Initialize the encoder and decoder optimizers with the above learning rate
    encoder_optimizer = FILL_IN
    decoder_optimizer = FILL_IN
    
    # Get n_iters training pairs
    # In this example, we are effectively doing SGD with batch size 1
    training_pairs = FILL_IN
    
    # The loss; either NLLLoss if you use log sigmoids or CrossEntropyLoss if you use logits
    criterion = FILL_IN

    for it in range(1, n_iters + 1):
        training_pair = training_pairs[it - 1]
        input_tensor = FILL_IN
        target_tensor = FILL_IN

        # Train on the input, target pair
        loss = FILL_IN
        
        # Update the total loss and the plot loss
        # We can plot and print at different granularities
        print_loss_total += loss
        plot_loss_total += loss

        if it % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(
                '%s (%d %d%%) %.4f' % (
                    timeSince(start, it / n_iters),
                    it, it / n_iters * 100, print_loss_avg)
            )

        if it % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

            showPlot(plot_losses)

Fill in the evaluation function using the hints below.

In [24]:
# For a certain input, get the predicted output sentence
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        # Transform the input sentence into a tensor
        input_tensor = tensorFromSentence(FILL_IN)
        input_length = input_tensor.size()[0]
        # Initilize the hidden and cell states of the LSTM
        encoder_hidden = FILL_IN
        encoder_cell = FILL_IN
        
        # Run the data through the LSTM word by word manually
        # At each step, feed in the input, the hidden state, and the cell state and calture the new hidden / cell states
        for it in range(input_length):
            encoder_output, (encoder_hidden, encoder_cell) = FILL_IN
        
        # Initialize the decoder input with the SOS token
        # This is y_0
        decoder_input = FILL_IN  # SOS

        # Initialize the decoder hidden and cell states with the final encoder hidden and cell states
        decoder_hidden = FILL_IN
        decoder_cell = FILL_IN

        decoded_words = []

        for jt in range(max_length):
            # As with the encoder run the \hat{y}_{t-1}, hidden, and state cells through the decoder
            # Capture the new hidden and cell states and the logits or log_softmax for the vocabulary
            decoder_output, (decoder_hidden, decoder_cell) = FILL_IN
            # Get the top y for the decoder, this will be the new \hat{y}_t which we can use at the next step
            _, topi = FILL_IN
            
            # Put logic so that if we get topi == EOS_token, we add this and break
            # Otherwise, we map the index topi to the word in output_lang via index2word
            FILL_IN

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [25]:
hidden_size = 256
cell_size = 256
# Initialize the encoder and decoder and run them through the trainIters function
encoder = FILL_IN
decoder = FILL_IN

trainIters(encoder, decoder, 75000, print_every=5000)

1m 4s (- 14m 58s) (5000 6%) 2.8336
2m 6s (- 13m 41s) (10000 13%) 2.3788
3m 16s (- 13m 5s) (15000 20%) 2.1190
4m 22s (- 12m 1s) (20000 26%) 1.9225
5m 25s (- 10m 51s) (25000 33%) 1.7266
6m 30s (- 9m 46s) (30000 40%) 1.5689
7m 36s (- 8m 41s) (35000 46%) 1.4171
8m 39s (- 7m 34s) (40000 53%) 1.3130
9m 42s (- 6m 28s) (45000 60%) 1.1897
10m 45s (- 5m 22s) (50000 66%) 1.0888
11m 49s (- 4m 17s) (55000 73%) 0.9790
12m 52s (- 3m 13s) (60000 80%) 0.9047
13m 55s (- 2m 8s) (65000 86%) 0.8080
47m 11s (- 3m 22s) (70000 93%) 0.7385
235m 55s (- 0m 0s) (75000 100%) 0.6681


Here we will do an evaluation.
Gather up n=7500 random pairs and for each pair get te BLEU score.
For this exercise, don't use BLUE based on 4-grams, use just 2-grams (you need to figure out how to specify this in $sentence\_bleu$ below). 
Investigate how this can be done with "sentence_bleu" in nltk - you can also find other implementations.
Print the average BLEU score after you've randomly drawn the sentences. You should be a BLEU of about 10 or slightly more.
Note that is is training BLEU, which is all I'd like you to get.

In [29]:
from nltk.translate.bleu_score import sentence_bleu

def evaluateRandomly(encoder, decoder, n=7500, debug=False):
    bleu_scores = []
    for i in range(n):
        # Randomly choose a pair of sentences
        pair = FILL_IN
        if debug:
            print('French Original: ', pair[0])
            print('English Reference: ', pair[1])
        # Leave out the <EOS> symbol
        # Run the source French sentence through the encoder-decoder and get the output_words
        output_words = FILL_IN
        
        # If <EOS> is at the end of output_words, remove it
        FILL_IN
                
        output_sentence = ' '.join(output_words)
        # Get the BLEU score based on 1 and 2 grams (words, bigrams); use 50% weight on each
        # Use pair[1] as the reference
        score = FILL_IN
        # Append the BLEU score you got to the list of BLEU scores you keep
        FILL_IN
        if debug:
            print('Candidate Translation: ', output_sentence)
            print('BLEU: ', score)
            print('')
    # Return the mean of the BLEU scores
    print('The mean BLEU score is: ', FILL_IN)

In [30]:
evaluateRandomly(encoder, decoder)

The mean BLEU score is:  10.651690951283388
